# Laboratorio: Operazioni avanzate con pandas

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

In questo laboratorio riprendiamo il dataset Rossmann utilizzato nel laboratorio precedente per vedere alcune operazioni più avanzate per la manipolazione dei dati in pandas.

Iniziamo come al solito importando le librerie necessarie con i rispettivi alias.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Caricamento dati

Riutilizziamo i due file con i dati Rossmann che abbiamo usato la scorsa volta:

- il file `rossmann-sales.csv` con i dati giornalieri delle vendite di 1.115 negozi nell'arco di luglio 2015
- il file `rossmann-stores.csv` con informazioni generali sugli stessi negozi

In [2]:
BASE_URL = "https://github.com/datascienceunibo/dialab2024/raw/main/Preprocessing_con_pandas"
SALES_URL = f"{BASE_URL}/rossmann-sales.csv"
STORES_URL = f"{BASE_URL}/rossmann-stores.csv"

La scorsa volta abbiamo caricato i file con la funzione `read_csv` lasciando tutte le impostazioni di default e correggendo a posteriori i tipi delle singole colonne.

Questa volta vediamo come impostare a priori i tipi delle colonne usando i parametri di `read_csv`. Iniziamo caricando il file `rossmann-sales.csv`.

Col parametro `dtype` possiamo indicare in un dizionario il tipo da attribuire a ciascuna singola colonna identificata per nome. Le colonne non indicate in `dtype` vengono interpretate come di consueto (come numeri se possibile, altrimenti come stringhe).

Usiamo `dtype` per indicare esplicitamente le colonne di tipo booleano e categorico, mentre la colonna `Date` di tipo data la indichiamo nel parametro `parse_dates` per indicare che bisogna interpretare le date scritte in formato standard.

In [3]:
sales_data = pd.read_csv(
    SALES_URL,
    dtype={
        "Open": bool,
        "Promo": bool,
        "StateHoliday": "category",
        "SchoolHoliday": bool,
    },
    parse_dates=["Date"],
)

Possiamo verificare che sia i dati che il formato delle colonne corrispondono a quelli che avevamo ottenuto la scorsa volta dopo le conversioni di tipo.

In [4]:
sales_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,True,True,0,True
1,2,5,2015-07-31,6064,625,True,True,0,True
2,3,5,2015-07-31,8314,821,True,True,0,True
3,4,5,2015-07-31,13995,1498,True,True,0,True
4,5,5,2015-07-31,4822,559,True,True,0,True
...,...,...,...,...,...,...,...,...,...
34560,1111,3,2015-07-01,3701,351,True,True,0,True
34561,1112,3,2015-07-01,10620,716,True,True,0,True
34562,1113,3,2015-07-01,8222,770,True,True,0,False
34563,1114,3,2015-07-01,27071,3788,True,True,0,False


In [5]:
sales_data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34565 entries, 0 to 34564
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Store          34565 non-null  int64         
 1   DayOfWeek      34565 non-null  int64         
 2   Date           34565 non-null  datetime64[ns]
 3   Sales          34565 non-null  int64         
 4   Customers      34565 non-null  int64         
 5   Open           34565 non-null  bool          
 6   Promo          34565 non-null  bool          
 7   StateHoliday   34565 non-null  category      
 8   SchoolHoliday  34565 non-null  bool          
dtypes: bool(3), category(1), datetime64[ns](1), int64(4)
memory usage: 1.5 MB


Per caricare il file `rossmann-stores.csv` utilizziamo di nuovo il parametro `dtype` per impostare i tipi delle colonne. Usiamo inoltre `index_col` per indicare il nome di una colonna che dovrà essere utilizzata come indice al posto di quello predefinito (numeri sequenziali da 0 a N-1).

In [6]:
stores_data = pd.read_csv(
    STORES_URL,
    index_col="Store",
    dtype={
        "StoreType": "category",
        "Assortment": "category",
        "Promo2": bool,
        "PromoInterval": "category",
    },
)

Anche in questo caso otteniamo i dati nella forma in cui li avevamo portati la scorsa volta.

In [7]:
stores_data

,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Store,,,,,,,,,
1,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2,a,a,570.0,11.0,2007.0,True,13.0,2010.0,"Jan,Apr,Jul,Oct"
3,a,a,14130.0,12.0,2006.0,True,14.0,2011.0,"Jan,Apr,Jul,Oct"
4,c,c,620.0,9.0,2009.0,False,NaN,NaN,NaN
5,a,a,29910.0,4.0,2015.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1111,a,a,1900.0,6.0,2014.0,True,31.0,2013.0,"Jan,Apr,Jul,Oct"
1112,c,c,1880.0,4.0,2006.0,False,NaN,NaN,NaN
1113,a,c,9260.0,NaN,NaN,False,NaN,NaN,NaN


In [8]:
stores_data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 1115 entries, 1 to 1115
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   StoreType                  1115 non-null   category
 1   Assortment                 1115 non-null   category
 2   CompetitionDistance        1112 non-null   float64 
 3   CompetitionOpenSinceMonth  761 non-null    float64 
 4   CompetitionOpenSinceYear   761 non-null    float64 
 5   Promo2                     1115 non-null   bool    
 6   Promo2SinceWeek            571 non-null    float64 
 7   Promo2SinceYear            571 non-null    float64 
 8   PromoInterval              571 non-null    category
dtypes: bool(1), category(3), float64(5)
memory usage: 57.6 KB


Applichiamo anche gli altri passaggi svolti nel precedente laboratorio:

- uniamo i due frame caricati per ottenere un unico frame che qui chiameremo `sales`, contenente sia le informazioni sulle vendite che quelle sui singoli negozi,
- selezioniamo da `sales` solo le righe relative a coppie giorno-negozio in cui il corrispondente negozio era aperto.

In [9]:
sales = pd.merge(sales_data, stores_data, left_on="Store", right_index=True)
sales = sales.loc[sales["Open"]].drop(columns=["Open"])

Prima di proseguire, scegliamo di ordinare le righe in primo luogo per data crescente e a parità per ID negozio crescente. Usiamo `sort_values` passando la lista con i nomi delle colonne.

In [10]:
sales = sales.sort_values(["Date", "Store"])

Per migliorare l'interpretabilità dei risultati successivi, sostituiamo inoltre ai numeri nella colonna `DayOfWeek` i nomi abbreviati dei giorni della settimana. Una volta definito un dizionario che mappa ciascuno dei valori distinti attualmente nella colonna al valore con cui si vuole sostituire...

In [11]:
index_to_day_of_week = {
    1: "Mon",
    2: "Tue",
    3: "Wed",
    4: "Thu",
    5: "Fri",
    6: "Sat",
    7: "Sun",
}

...possiamo usare il metodo `map` per ottenere una colonna con i valori sostituiti, che convertiamo poi a categorica.

In [12]:
sales["DayOfWeek"] = sales["DayOfWeek"].map(index_to_day_of_week).astype("category")

Stampiamo il frame risultante allo stato attuale.

In [13]:
sales

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
33450,1,Wed,2015-07-01,5223,562,True,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
33451,2,Wed,2015-07-01,6431,786,True,0,False,a,a,570.0,11.0,2007.0,True,13.0,2010.0,"Jan,Apr,Jul,Oct"
33452,3,Wed,2015-07-01,8311,772,True,0,True,a,a,14130.0,12.0,2006.0,True,14.0,2011.0,"Jan,Apr,Jul,Oct"
33453,4,Wed,2015-07-01,11216,1359,True,0,False,c,c,620.0,9.0,2009.0,False,NaN,NaN,NaN
33454,5,Wed,2015-07-01,6253,717,True,0,False,a,a,29910.0,4.0,2015.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,1111,Fri,2015-07-31,5723,422,True,0,True,a,a,1900.0,6.0,2014.0,True,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,Fri,2015-07-31,9626,767,True,0,True,c,c,1880.0,4.0,2006.0,False,NaN,NaN,NaN
1112,1113,Fri,2015-07-31,7289,720,True,0,True,a,c,9260.0,NaN,NaN,False,NaN,NaN,NaN
1113,1114,Fri,2015-07-31,27508,3745,True,0,True,a,c,870.0,NaN,NaN,False,NaN,NaN,NaN


Riepiloghiamo i significati delle colonne:

- `Store`: identificatore del negozio, un intero da 1 a 1.115
- `DayOfWeek`: giorno della settimana
- `Date`: data a cui si riferiscono i dati
- `Sales`: ricavo totale del negozio nella giornata _(valuta non indicata, assumeremo in Euro)_
- `Customers`: numero di clienti del negozio nella giornata
- `Promo`: indica se una promozione era in corso nel negozio (1 = in corso, 0 = non in corso)
- `StateHoliday`: indica se nella data c'era una festività e di quale tipo (`a` = festa nazionale, `b` = Pasqua, `c` = Natale, 0 = nessuna festività)
- `SchoolHoliday`: indica se nella data le scuole erano chiuse (1 = chiuse, 0 = aperte)
- `StoreType`: lettera che distingue la tipologia di negozio (`a`, `b`, `c` o `d`)
- `Assortment`: livello di assortimento dei prodotti nel negozio (`a` = base, `b` = extra, `c` = esteso)
- `CompetitionDistance`: distanza in metri dal punto vendita concorrente più vicino
- `CompetitionOpenSinceMonth` e `CompetitionOpenSinceYear`: mese e anno in cui il punto vendita concorrente più vicino è stato aperto
- `Promo2`: indica se il negozio prende parte a una promozione continuativa (0 = no, 1 = sì)
- `Promo2SinceWeek` e `Promo2SinceYear`: numero di settimana e anno a partire dai quali il negozio prende parte alla `Promo2`
- `PromoInterval`: mesi dell'anno in cui il negozio avvia la `Promo2`

### Esercizio 1: Ripasso

- **(1a)** Estrarre da `sales` le righe relative ai casi in cui ci sono stati almeno 4.000 clienti in un giorno.
- **(1b)** Ottenere il ricavo medio giornaliero di un negozio nelle sole domeniche.
- **(1c)** Contare in quante righe il valore di `CompetitionDistance` è mancante.
- **(1d)** Ottenere una serie col numero di negozi aperti in ciascuna data.

In [14]:
# 1a
sales.loc[sales["Customers"] >= 4000]

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
34218,769,Wed,2015-07-01,14985,4424,True,0,True,b,b,840.0,NaN,NaN,True,48.0,2012.0,"Jan,Apr,Jul,Oct"
33067,733,Thu,2015-07-02,16883,4180,True,0,True,b,b,860.0,10.0,1999.0,False,NaN,NaN,NaN
33103,769,Thu,2015-07-02,14635,4582,True,0,True,b,b,840.0,NaN,NaN,True,48.0,2012.0,"Jan,Apr,Jul,Oct"
31481,262,Fri,2015-07-03,23253,4051,True,0,False,b,a,1180.0,5.0,2013.0,False,NaN,NaN,NaN
31952,733,Fri,2015-07-03,17175,4054,True,0,True,b,b,860.0,10.0,1999.0,False,NaN,NaN,NaN
31988,769,Fri,2015-07-03,14553,4300,True,0,True,b,b,840.0,NaN,NaN,True,48.0,2012.0,"Jan,Apr,Jul,Oct"
30873,769,Sat,2015-07-04,12477,4136,False,0,False,b,b,840.0,NaN,NaN,True,48.0,2012.0,"Jan,Apr,Jul,Oct"
29251,262,Sun,2015-07-05,30255,4762,False,0,False,b,a,1180.0,5.0,2013.0,False,NaN,NaN,NaN
21446,262,Sun,2015-07-12,32271,4623,False,0,False,b,a,1180.0,5.0,2013.0,False,NaN,NaN,NaN
16342,733,Fri,2015-07-17,15016,4013,True,0,True,b,b,860.0,10.0,1999.0,False,NaN,NaN,NaN


In [15]:
# 1b
sales.loc[sales["DayOfWeek"] == "Sun", "Sales"].mean()

8764.170542635658

In [16]:
# 1c
sales["CompetitionDistance"].isna().sum()

81

In [17]:
# 1d
sales["Date"].value_counts()

Date
2015-07-01    1115
2015-07-07    1114
2015-07-02    1114
2015-07-09    1114
2015-07-08    1114
2015-07-10    1114
2015-07-06    1114
2015-07-04    1114
2015-07-03    1114
2015-07-30    1113
2015-07-20    1113
2015-07-29    1113
2015-07-28    1113
2015-07-27    1113
2015-07-25    1113
2015-07-24    1113
2015-07-23    1113
2015-07-22    1113
2015-07-21    1113
2015-07-31    1113
2015-07-18    1113
2015-07-15    1113
2015-07-14    1113
2015-07-13    1113
2015-07-11    1113
2015-07-17    1112
2015-07-16    1112
2015-07-05      33
2015-07-19      32
2015-07-26      32
2015-07-12      32
Name: count, dtype: int64

## Indici a più livelli

Gli indici di pandas associano un'etichetta identificativa a ciascun elemento di una serie e a ciascuna riga e colonna di un DataFrame.

Finora abbiamo visto indici dove **ciascuna etichetta** è un singolo valore, ad es. un numero o una stringa, ma in alcuni casi è utile che ogni etichetta sia una **tupla di valori**.

Si pensi come analogia ad una tabella di un database relazionale con una chiave primaria composta da più colonne: ciascuna riga è identificata dalla combinazione dei valori di tali colonne. Tali combinazioni devono essere univoche, ma i valori all'interno delle singole colonne si possono ripetere: ad es. in una tabella con chiave primaria `(A, B)` non posso avere più righe con `A=1, B=2`, ma posso averne una con `A=1, B=2` e una con `A=1, B=3`.

Dato che ogni riga del nostro frame `sales` rappresenta le informazioni relative ad uno specifico negozio in una specifica data, si può dire che la combinazione di data e ID negozio identifichi ciascuna riga. Ha quindi senso che ciascuna riga sia etichettata dalla combinazione delle colonne `Date` e `Store`.

Quando impostiamo l'indice delle righe di un frame (ad es. col parametro `index_col` di `read_csv` o col metodo `set_index`) possiamo passare una sequenza di _N_ nomi di colonne per creare un _indice a più livelli_ (o _indice gerarchico_), dove l'indice è composto da _N_ sequenze di valori e ciascuna riga è così etichettata da una tupla di _N_ valori.

Trasformiamo quindi il frame `sales` impostando come indice la combinazione delle colonne `Date` e `Store`.

In [18]:
sales = sales.set_index(["Date", "Store"])

Vediamo come viene visualizzato ora il frame `sales`.

In [19]:
sales

DayOfWeek  Sales  Customers  Promo StateHoliday  \
Date       Store                                                   
2015-07-01 1           Wed   5223        562   True            0   
           2           Wed   6431        786   True            0   
           3           Wed   8311        772   True            0   
           4           Wed  11216       1359   True            0   
           5           Wed   6253        717   True            0   
...                    ...    ...        ...    ...          ...   
2015-07-31 1111        Fri   5723        422   True            0   
           1112        Fri   9626        767   True            0   
           1113        Fri   7289        720   True            0   
           1114        Fri  27508       3745   True            0   
           1115        Fri   8680        538   True            0   

                  SchoolHoliday StoreType Assortment  CompetitionDistance  \
Date       Store                                                            
2015-07-01 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
           4              False         c          c                620.0   
           5              False         a          a              29910.0   
...                         ...       ...        ...                  ...   
2015-07-31 1111            True         a          a               1900.0   
           1112            True         c          c               1880.0   
           1113            True         a          c               9260.0   
           1114            True         a          c                870.0   
           1115            True         d          c               5350.0   

                  CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
Date       Store                                                                
2015-07-01 1                            9.0                    2008.0   False   
           2                           11.0                    2007.0    True   
           3                           12.0                    2006.0    True   
           4                            9.0                    2009.0   False   
           5                            4.0                    2015.0   False   
...                                     ...                       ...     ...   
2015-07-31 1111                         6.0                    2014.0    True   
           1112                         4.0                    2006.0   False   
           1113                         NaN                       NaN   False   
           1114                         NaN                       NaN   False   
           1115                         NaN                       NaN    True   

                  Promo2SinceWeek  Promo2SinceYear     PromoInterval  
Date       Store                                                      
2015-07-01 1                  NaN              NaN               NaN  
           2                 13.0           2010.0   Jan,Apr,Jul,Oct  
           3                 14.0           2011.0   Jan,Apr,Jul,Oct  
           4                  NaN              NaN               NaN  
           5                  NaN              NaN               NaN  
...                           ...              ...               ...  
2015-07-31 1111              31.0           2013.0   Jan,Apr,Jul,Oct  
           1112               NaN              NaN               NaN  
           1113               NaN              NaN               NaN  
           1114               NaN              NaN               NaN  
           1115              22.0           2012.0  Mar,Jun,Sept,Dec  

[30188 rows x 15 columns]

`Date` e `Store`, che prima erano colonne vere e proprie, sono diventate i due livelli dell'indice delle righe. I valori dei livelli più a sinistra (qui `Date`) sono riportati solo una volta nei casi in cui si ripetono in più righe consecutive.

L'indice delle righe è ora un oggetto `MultiIndex`, composto da una sequenza di tuple (data, ID negozio).

In [20]:
sales.index

MultiIndex([('2015-07-01',    1),
            ('2015-07-01',    2),
            ('2015-07-01',    3),
            ('2015-07-01',    4),
            ('2015-07-01',    5),
            ('2015-07-01',    6),
            ('2015-07-01',    7),
            ('2015-07-01',    8),
            ('2015-07-01',    9),
            ('2015-07-01',   10),
            ...
            ('2015-07-31', 1106),
            ('2015-07-31', 1107),
            ('2015-07-31', 1108),
            ('2015-07-31', 1109),
            ('2015-07-31', 1110),
            ('2015-07-31', 1111),
            ('2015-07-31', 1112),
            ('2015-07-31', 1113),
            ('2015-07-31', 1114),
            ('2015-07-31', 1115)],
           names=['Date', 'Store'], length=30188)

Per estrarre la sequenza di valori per un singolo livello si può usare il metodo `get_level_values` indicando il nome del livello o la posizione.

In [21]:
sales.index.get_level_values("Date")  # oppure (0)

DatetimeIndex(['2015-07-01', '2015-07-01', '2015-07-01', '2015-07-01',
               '2015-07-01', '2015-07-01', '2015-07-01', '2015-07-01',
               '2015-07-01', '2015-07-01',
               ...
               '2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31'],
              dtype='datetime64[ns]', name='Date', length=30188, freq=None)

### Selezione su indici multi-livello

La selezione per etichette con `loc` su frame con indici a più livelli può essere effettuata come in quelli normali, tenendo presente che le righe questa volta sono identificate da tuple invece che da singoli valori.

Internamente i valori del livello `Date` sono di tipo data, ma pandas consente di esprimerli come stringhe nel formato standard `YYYY-MM-DD` usato anche per visualizzarle.

Ad esempio per selezionare la prima riga, relativa al 1° luglio 2015 e al negozio 1, scrivo:

In [22]:
sales.loc[("2015-07-01", 1)]

DayOfWeek                       Wed
Sales                          5223
Customers                       562
Promo                          True
StateHoliday                      0
SchoolHoliday                 False
StoreType                         c
Assortment                        a
CompetitionDistance          1270.0
CompetitionOpenSinceMonth       9.0
CompetitionOpenSinceYear     2008.0
Promo2                        False
Promo2SinceWeek                 NaN
Promo2SinceYear                 NaN
PromoInterval                   NaN
Name: (2015-07-01 00:00:00, 1), dtype: object

Se da qui voglio selezionare una cella specifica, ad es. la colonna `Sales`:

In [23]:
sales.loc[("2015-07-01", 1), "Sales"]

5223

Una possibilità ulteriore data dagli indici a più livelli consiste nell'indicare un'etichetta "parziale" che contiene solo i valori di alcuni livelli per selezionare tutte le righe con etichette che combaciano.

Nello specifico, in `sales` possiamo selezionare tutte le righe con una certa data.

In [24]:
sales.loc["2015-07-01"]

,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Store,,,,,,,,,,,,,,,
1,Wed,5223,562,True,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2,Wed,6431,786,True,0,False,a,a,570.0,11.0,2007.0,True,13.0,2010.0,"Jan,Apr,Jul,Oct"
3,Wed,8311,772,True,0,True,a,a,14130.0,12.0,2006.0,True,14.0,2011.0,"Jan,Apr,Jul,Oct"
4,Wed,11216,1359,True,0,False,c,c,620.0,9.0,2009.0,False,NaN,NaN,NaN
5,Wed,6253,717,True,0,False,a,a,29910.0,4.0,2015.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,Wed,3701,351,True,0,True,a,a,1900.0,6.0,2014.0,True,31.0,2013.0,"Jan,Apr,Jul,Oct"
1112,Wed,10620,716,True,0,True,c,c,1880.0,4.0,2006.0,False,NaN,NaN,NaN
1113,Wed,8222,770,True,0,False,a,c,9260.0,NaN,NaN,False,NaN,NaN,NaN


Notare che, avendo selezionato un singolo valore del livello `Date`, questo viene rimosso dal frame risultante, che contiene solo il livello `Store` per distinguere le righe. Questo è analogo al fatto che selezionando una singola riga (o colonna) di un frame ottengo una serie il cui indice è quello delle colonne (o righe) del frame.

Possiamo anche selezionare una lista o un intervallo di date (in questo caso il livello `Date` viene mantenuto per distinguerle).

In [25]:
sales.loc["2015-07-01":"2015-07-03"]

DayOfWeek  Sales  Customers  Promo StateHoliday  \
Date       Store                                                   
2015-07-01 1           Wed   5223        562   True            0   
           2           Wed   6431        786   True            0   
           3           Wed   8311        772   True            0   
           4           Wed  11216       1359   True            0   
           5           Wed   6253        717   True            0   
...                    ...    ...        ...    ...          ...   
2015-07-03 1111        Fri   4596        392   True            0   
           1112        Fri   9285        717   True            0   
           1113        Fri   6942        670   True            0   
           1114        Fri  23750       3675   True            0   
           1115        Fri   7412        478   True            0   

                  SchoolHoliday StoreType Assortment  CompetitionDistance  \
Date       Store                                                            
2015-07-01 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
           4              False         c          c                620.0   
           5              False         a          a              29910.0   
...                         ...       ...        ...                  ...   
2015-07-03 1111            True         a          a               1900.0   
           1112            True         c          c               1880.0   
           1113           False         a          c               9260.0   
           1114           False         a          c                870.0   
           1115           False         d          c               5350.0   

                  CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
Date       Store                                                                
2015-07-01 1                            9.0                    2008.0   False   
           2                           11.0                    2007.0    True   
           3                           12.0                    2006.0    True   
           4                            9.0                    2009.0   False   
           5                            4.0                    2015.0   False   
...                                     ...                       ...     ...   
2015-07-03 1111                         6.0                    2014.0    True   
           1112                         4.0                    2006.0   False   
           1113                         NaN                       NaN   False   
           1114                         NaN                       NaN   False   
           1115                         NaN                       NaN    True   

                  Promo2SinceWeek  Promo2SinceYear     PromoInterval  
Date       Store                                                      
2015-07-01 1                  NaN              NaN               NaN  
           2                 13.0           2010.0   Jan,Apr,Jul,Oct  
           3                 14.0           2011.0   Jan,Apr,Jul,Oct  
           4                  NaN              NaN               NaN  
           5                  NaN              NaN               NaN  
...                           ...              ...               ...  
2015-07-03 1111              31.0           2013.0   Jan,Apr,Jul,Oct  
           1112               NaN              NaN               NaN  
           1113               NaN              NaN               NaN  
           1114               NaN              NaN               NaN  
           1115              22.0           2012.0  Mar,Jun,Sept,Dec  

[3343 rows x 15 columns]

Per selezionare righe in base al valore del secondo livello dell'indice (`Store`) invece che del primo, possiamo indicare come valore del primo livello `slice(None)`.

_`slice(None)` equivale alla notazione `:` di intervallo senza inizio o fine, che per la sintassi di Python non si può però usare all'interno di parentesi tonde. Nella [guida di pandas](https://pandas.pydata.org/docs/user_guide/advanced.html) sono riportate sintassi alternative._

Selezioniamo ad esempio tutte le righe relative al negozio con ID 1.

In [26]:
sales.loc[(slice(None), 1), :]

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,Store,,,,,,,,,,,,,,,
2015-07-01,1,Wed,5223,562,True,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-02,1,Thu,5558,573,True,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-03,1,Fri,4665,538,True,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-04,1,Sat,4797,560,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-06,1,Mon,4359,540,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-07,1,Tue,3650,485,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-08,1,Wed,3797,485,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-09,1,Thu,3897,480,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN
2015-07-10,1,Fri,3808,449,False,0,False,c,a,1270.0,9.0,2008.0,False,NaN,NaN,NaN


### Esercizio 2: Indici a più livelli

- **(2a)** Qual è stato il numero di clienti nel negozio con ID 42 il 1° luglio 2015?
- **(2b)** Qual è stato il ricavo totale di tutti i negozi il 2 luglio 2015?
- **(2c)** Qual è stato il ricavo totale dei negozi con ID 2, 15 e 18 il 3 luglio 2015?

In [27]:
# 2a
sales.loc[("2015-07-01", 42), "Customers"]

1203

In [28]:
# 2b
sales.loc["2015-07-02", "Sales"].sum()

8446293

In [29]:
# 2c
sales.loc[("2015-07-03", [2, 15, 18]), "Sales"].sum()

20202

## Raggruppamento

Abbiamo visto come calcolare statistiche (somma, media, ...) ed eseguire altre operazioni sugli interi DataFrame.

È però spesso opportuno estrarre statistiche simili oppure compiere operazioni su gruppi di dati separatamente.

Come calcolare, ad esempio, qual è la media dei ricavi per giorno della settimana?

Possiamo ottenere questa informazione con i metodi visti finora, ma in modo non del tutto immediato...

In [30]:
# per ogni possibile valore distinto di giorno della settimana
for dow in sales["DayOfWeek"].unique():
    # estraggo i relativi valori di Sales dal frame e calcolo la media
    mean_sales_on_dow = sales.loc[sales["DayOfWeek"] == dow, "Sales"].mean()
    # stampo il giorno della settimana e la media
    print(f"{dow:>4}: {mean_sales_on_dow:6.2f}")

 Wed: 7053.12
 Thu: 7050.71
 Fri: 7232.43
 Sat: 5566.35
 Sun: 8764.17
 Mon: 8180.63
 Tue: 7007.63


Per esigenze come questa, pandas offre il modo per partizionare un frame in due o più **gruppi** di righe e applicare gruppo per gruppo operazioni che solitamente sono applicate all'intero frame.

Per ottenere un DataFrame partizionato usiamo il metodo `groupby` indicando un criterio di partizionamento, che nel caso più semplice è il nome di una colonna o di un livello dell'indice. Usiamo ad esempio il giorno della settimana `DayOfWeek` per creare un gruppo per ciascun giorno della settimana.

Sull'oggetto `DataFrameGroupBy` risultante è possibile accedere a diversi attributi e metodi, ad esempio `ngroups` indica il numero di gruppi ottenuti.

In [31]:
sales.groupby("DayOfWeek").ngroups

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3658637923.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek").ngroups


7

Il dizionario `groups` riporta invece i gruppi formati: le chiavi sono i valori distinti della colonna di raggruppamento, mentre il valore associato a ciascuna è la lista delle etichette delle righe contenute nel relativo gruppo.

In [32]:
list(sales.groupby("DayOfWeek").groups.keys())

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\1313029192.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  list(sales.groupby("DayOfWeek").groups.keys())


['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']

Col metodo `size` otteniamo il numero di righe in ciascun gruppo. Il risultato è in pratica lo stesso che si otterrebbe (ordine a parte) con `sales["DayOfWeek"].value_counts()`

In [33]:
sales.groupby("DayOfWeek").size()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2702253015.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek").size()


DayOfWeek
Fri    5566
Mon    4453
Sat    4453
Sun     129
Thu    5566
Tue    4453
Wed    5568
dtype: int64

### Operazioni sui gruppi

Sul frame partizionato è possibile chiamare i metodi di aggregazione già visti per ottenere **statistiche divise per gruppi**.

Invochiamo ad esempio il metodo `mean` per il calcolo della media, specificando `numeric_only=True` per escludere le colonne non numeriche.

In [34]:
sales.groupby("DayOfWeek").mean(numeric_only=True)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\160502643.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek").mean(numeric_only=True)


,Sales,Customers,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
DayOfWeek,,,,,,,,,,
Fri,7232.429213,759.956342,0.599892,0.500180,5400.385516,7.228218,2008.671492,0.511858,23.578800,2011.763075
Mon,8180.631709,818.630586,0.499888,0.513137,5402.107633,7.228036,2008.670944,0.512014,23.581579,2011.763158
Sat,5566.347631,606.321356,0.000000,0.004940,5402.107633,7.228036,2008.670944,0.512014,23.581579,2011.763158
Sun,8764.170543,1458.480620,0.000000,0.007752,13052.015504,7.550000,2010.050000,0.279070,18.222222,2012.888889
Thu,7050.713259,742.966223,0.599892,0.500180,5400.385516,7.228218,2008.671492,0.511858,23.578800,2011.763075
Tue,7007.631709,734.454750,0.499888,0.513137,5402.107633,7.228036,2008.670944,0.512014,23.581579,2011.763158
Wed,7053.122126,733.591774,0.600036,0.461746,5402.667027,7.227368,2008.670526,0.512033,23.584356,2011.763241


Invece di una serie con le medie globali di ciascuna colonna, si ottiene un frame con **una riga per ogni gruppo** che riporta le medie all'interno rispettivo gruppo.

Dal frame partizionato possiamo anche selezionare una singola colonna per ottenere un oggetto `SeriesGroupBy`, ovvero una serie partizionata. Su questa possiamo invocare i soliti metodi di riduzione per ottenere statistiche per gruppi limitate alla colonna selezionata. Allo stesso modo possiamo selezionare una lista di colonne.

Possiamo così ottenere la media dei ricavi per giorno della settimana che avevamo ottenuto sopra, ma in modo più semplice.

In [35]:
sales.groupby("DayOfWeek")["Sales"].mean()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\1078311500.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek")["Sales"].mean()


DayOfWeek
Fri    7232.429213
Mon    8180.631709
Sat    5566.347631
Sun    8764.170543
Thu    7050.713259
Tue    7007.631709
Wed    7053.122126
Name: Sales, dtype: float64

Possiamo usare `describe` per ottenere tutte le statistiche comuni per ogni gruppo.

In [36]:
sales.groupby("DayOfWeek")["Sales"].describe()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3996018584.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek")["Sales"].describe()


,count,mean,std,min,25%,50%,75%,max
DayOfWeek,,,,,,,,
Fri,5566.0,7232.429213,2782.014062,1012.0,5332.25,6742.0,8547.75,27508.0
Mon,4453.0,8180.631709,3294.695447,2115.0,5821.00,7610.0,9848.00,28156.0
Sat,4453.0,5566.347631,2574.491468,708.0,3784.00,5213.0,6795.00,23091.0
Sun,129.0,8764.170543,6407.655057,1292.0,3916.00,7361.0,12040.00,32547.0
Thu,5566.0,7050.713259,2649.212644,2002.0,5243.25,6590.0,8276.00,24898.0
Tue,4453.0,7007.631709,2843.267707,1958.0,5009.00,6485.0,8403.00,25518.0
Wed,5568.0,7053.122126,2784.489329,1737.0,5176.25,6559.0,8332.00,27071.0


Se si vuole cambiare il formato della tabella mettendo una statistica per riga e un gruppo per colonna, si può come per gli array NumPy usare l'operatore `.T` per trasporre un DataFrame.

In [37]:
sales.groupby("DayOfWeek")["Sales"].describe().T

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3495664229.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek")["Sales"].describe().T


DayOfWeek,Fri,Mon,Sat,Sun,Thu,Tue,Wed
count,5566.000000,4453.000000,4453.000000,129.000000,5566.000000,4453.000000,5568.000000
mean,7232.429213,8180.631709,5566.347631,8764.170543,7050.713259,7007.631709,7053.122126
std,2782.014062,3294.695447,2574.491468,6407.655057,2649.212644,2843.267707,2784.489329
min,1012.000000,2115.000000,708.000000,1292.000000,2002.000000,1958.000000,1737.000000
25%,5332.250000,5821.000000,3784.000000,3916.000000,5243.250000,5009.000000,5176.250000
50%,6742.000000,7610.000000,5213.000000,7361.000000,6590.000000,6485.000000,6559.000000
75%,8547.750000,9848.000000,6795.000000,12040.000000,8276.000000,8403.000000,8332.000000
max,27508.000000,28156.000000,23091.000000,32547.000000,24898.000000,25518.000000,27071.000000


Se siamo interessati ad un insieme specifico di statistiche, possiamo usare il metodo `agg` (disponibile anche per serie e frame senza `groupby`) indicando una lista con i nomi delle statistiche.

Otteniamo ad esempio un frame che riporta per ogni giorno della settimana il ricavo complessivo e quello medio per giorno e negozio.

In [38]:
sales.groupby("DayOfWeek")["Sales"].agg(["sum", "mean"])

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2372250661.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek")["Sales"].agg(["sum", "mean"])


,sum,mean
DayOfWeek,,
Fri,40255701,7232.429213
Mon,36428353,8180.631709
Sat,24786946,5566.347631
Sun,1130578,8764.170543
Thu,39244270,7050.713259
Tue,31204984,7007.631709
Wed,39271784,7053.122126


Nei casi in cui sia le colonne che le statistiche selezionate sono più d'una, si ottiene un frame con un indice delle colonne a più livelli: il primo per le colonne e il secondo per le statistiche.

In [39]:
sales.groupby("DayOfWeek")[["Sales", "Customers"]].agg(["mean", "std"])

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3963646109.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek")[["Sales", "Customers"]].agg(["mean", "std"])


Sales                 Customers             
                  mean          std         mean          std
DayOfWeek                                                    
Fri        7232.429213  2782.014062   759.956342   376.487010
Mon        8180.631709  3294.695447   818.630586   382.568461
Sat        5566.347631  2574.491468   606.321356   351.291072
Sun        8764.170543  6407.655057  1458.480620  1155.975441
Thu        7050.713259  2649.212644   742.966223   366.865006
Tue        7007.631709  2843.267707   734.454750   369.781209
Wed        7053.122126  2784.489329   733.591774   373.008639

Anche altre operazioni viste finora possono essere applicate gruppo per gruppo. Ad esempio i metodi `head` e `tail` restituiscono le prime/ultime _N_ righe di ciascun gruppo. Questo è in pratica un altro modo per selezionare alcune righe specifiche dal frame d'origine.

In [40]:
sales.groupby("DayOfWeek").head(3)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\4132971036.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby("DayOfWeek").head(3)


DayOfWeek  Sales  Customers  Promo StateHoliday  \
Date       Store                                                   
2015-07-01 1           Wed   5223        562   True            0   
           2           Wed   6431        786   True            0   
           3           Wed   8311        772   True            0   
2015-07-02 1           Thu   5558        573   True            0   
           2           Thu   6189        921   True            0   
           3           Thu   7450        701   True            0   
2015-07-03 1           Fri   4665        538   True            0   
           2           Fri   4838       1029   True            0   
           3           Fri   8149        742   True            0   
2015-07-04 1           Sat   4797        560  False            0   
           2           Sat   4475       1147  False            0   
           3           Sat   4482        456  False            0   
2015-07-05 85          Sun  12070       1734  False            0   
           122         Sun   3092        411  False            0   
           209         Sun   2447        245  False            0   
2015-07-06 1           Mon   4359        540  False            0   
           2           Mon   4132        523  False            0   
           3           Mon   6736        734  False            0   
2015-07-07 1           Tue   3650        485  False            0   
           2           Tue   4313        531  False            0   
           3           Tue   6180        657  False            0   

                  SchoolHoliday StoreType Assortment  CompetitionDistance  \
Date       Store                                                            
2015-07-01 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
2015-07-02 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
2015-07-03 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
2015-07-04 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3              False         a          a              14130.0   
2015-07-05 85             False         b          a               1870.0   
           122            False         a          c              58260.0   
           209            False         a          c              11680.0   
2015-07-06 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   
2015-07-07 1              False         c          a               1270.0   
           2              False         a          a                570.0   
           3               True         a          a              14130.0   

                  CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
Date       Store                                                                
2015-07-01 1                            9.0                    2008.0   False   
           2                           11.0                    2007.0    True   
           3                           12.0                    2006.0    True   
2015-07-02 1                            9.0                    2008.0   False   
           2                           11.0                    2007.0    True   
           3                           12.0                    2006.0    True   
2015-07-03 1                    

### Raggruppamento su valori derivati

Oltre che su una colonna, i gruppi possono essere creati in base ad **altre serie con indice uguale** a quello del DataFrame, ad es. a variabili **derivate dalle colonne esistenti**.

Ipotizziamo ad esempio di voler dividere i negozi in fasce per distanza dalla concorrenza ed ottenere statistiche suddivise per tali fasce. Usando ad es. `qcut` ottengo una serie che ha lo stesso indice a due livelli di `sales` che associa una fascia di distanza a ciascuna etichetta.

In [41]:
pd.qcut(sales["CompetitionDistance"], 3)

Date        Store
2015-07-01  1         (1170.0, 4520.0]
            2         (19.999, 1170.0]
            3        (4520.0, 75860.0]
            4         (19.999, 1170.0]
            5        (4520.0, 75860.0]
                           ...        
2015-07-31  1111      (1170.0, 4520.0]
            1112      (1170.0, 4520.0]
            1113     (4520.0, 75860.0]
            1114      (19.999, 1170.0]
            1115     (4520.0, 75860.0]
Name: CompetitionDistance, Length: 30188, dtype: category
Categories (3, interval[float64, right]): [(19.999, 1170.0] < (1170.0, 4520.0] < (4520.0, 75860.0]]

Questa serie può essere usata suddividere `sales` in gruppi ed estrarre statistiche per gruppo, ad es. la media giornaliera di ricavi e clienti.

In [42]:
sales.groupby(pd.qcut(sales["CompetitionDistance"], 3))[["Sales", "Customers"]].mean()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2831124815.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby(pd.qcut(sales["CompetitionDistance"], 3))[["Sales", "Customers"]].mean()


,Sales,Customers
CompetitionDistance,,
"(19.999, 1170.0]",7110.328021,857.941926
"(1170.0, 4520.0]",6898.946077,708.520060
"(4520.0, 75860.0]",7103.049110,646.495899


### Chiavi di raggruppamento multiple

Possiamo combinare più criteri di raggruppamento dei tipi visti sopra per ottenere gruppi più specifici, uno per ogni combinazione possibile.

Creiamo ad esempio un raggruppamento sulle due colonne categoriche `StoreType` e `Assortment` che indicano la tipologia di negozio e di assortimento di prodotti, passando a `groupby` una lista con i due nomi.

Invocando i metodi visti sopra per ottenere statistiche per gruppi, otteniamo un frame con indice delle righe a più livelli, dove ogni livello corrisponde ad uno dei criteri di raggruppamento passati a `groupby`.

In [43]:
sales.groupby(["StoreType", "Assortment"])[["Sales", "Customers"]].mean()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2273490590.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby(["StoreType", "Assortment"])[["Sales", "Customers"]].mean()


Sales    Customers
StoreType Assortment                           
a         a            6417.942760   717.797367
          b                    NaN          NaN
          c            7776.345052   838.057647
b         a           11412.990783  1800.797235
          b            9717.379928  2344.906810
          c           17526.387097  3059.806452
c         a            6775.873497   794.636845
          b                    NaN          NaN
          c            7170.213876   762.871674
d         a            6486.686994   566.006647
          b                    NaN          NaN
          c            7368.128110   613.675185

### Esercizio 3: Raggruppamento

- **(3a)** Ricavare il numero di clienti medio giornaliero suddiviso per giorno della settimana e per casi in cui era applicata o meno una promozione (`Promo`)
- **(3b)** Ricavare le medie giornaliere di ricavo e numero di clienti suddivise tra weekend (sabato e domenica) e giorni infrasettimanali.
  - su una serie `X` usare `X.isin([...])` per verificare quali valori della serie sono inclusi in una lista data
- **(3c)** In quale data il ricavo totale di tutti i negozi è stato maggiore?
- **(3d)** Per quante volte ciascun negozio è stato quello con più ricavi in una certa data? _(il negozio 1114 lo è stato 18 volte, il negozio 842 lo è stato 6 volte, ecc.)_

In [44]:
# 3a
sales.groupby(["DayOfWeek", "Promo"])["Customers"].mean()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3886654935.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby(["DayOfWeek", "Promo"])["Customers"].mean()


DayOfWeek  Promo
Fri        False     697.350696
           True      801.712189
Mon        False     749.848226
           True      887.443845
Sat        False     606.321356
           True             NaN
Sun        False    1458.480620
           True             NaN
Thu        False     693.390211
           True      776.031746
Tue        False     665.662775
           True      803.277628
Wed        False     665.228559
           True      779.160431
Name: Customers, dtype: float64

In [45]:
# 3b
sales.groupby(sales["DayOfWeek"].isin(["Sat", "Sun"]))[["Sales", "Customers"]].mean()

,Sales,Customers
DayOfWeek,,
False,7279.742717,756.299110
True,5656.378001,630.312746


In [46]:
# 3c
sales.groupby("Date")["Sales"].sum().idxmax()

Timestamp('2015-07-13 00:00:00')

In [47]:
# 3d
(
    # ordino il frame per ricavi discendenti
    sales.sort_values("Sales", ascending=False)
    # partiziono il frame per date
    .groupby("Date")
    # per ogni gruppo prendo la prima riga
    .head(1)
    # dal frame risultante prendo i valori del livello Store dell'indice
    .index.get_level_values("Store")
    # conto le occorrenze di ciascun valore
    .value_counts()
)

Store
1114    18
842      6
262      4
876      2
817      1
Name: count, dtype: int64

## Pivoting

Una _tabella pivot_ è in generale una tabella che contiene statistiche aggregate di valori provenienti da una tabella "sorgente" suddivise per gruppi.

In questo senso i frame che abbiamo ottenuto sopra tramite `groupby` possono definirsi delle tabelle pivot, dove ogni riga corrisponde ad un gruppo di dati e ogni colonna ad una statistica calcolata su ciascun gruppo.

Potremmo però voler organizzare i dati nella tabella in modo differente. Prendiamo ad esempio il ricavo medio giornaliero per tipo di negozio e di assortimento.

In [48]:
sales.groupby(["StoreType", "Assortment"])["Sales"].mean()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\561820688.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby(["StoreType", "Assortment"])["Sales"].mean()


StoreType  Assortment
a          a              6417.942760
           b                      NaN
           c              7776.345052
b          a             11412.990783
           b              9717.379928
           c             17526.387097
c          a              6775.873497
           b                      NaN
           c              7170.213876
d          a              6486.686994
           b                      NaN
           c              7368.128110
Name: Sales, dtype: float64

Per confrontare meglio i valori, potremmo volerli disposti in una tabella dove ad ogni riga corrisponde uno `StoreType` e ad ogni colonna un `Assortment`, o viceversa.

pandas offre operazioni sia per trasformare serie o frame esistenti basate sugli indici a più livelli, sia operazioni per creare tabelle pivot personalizzate direttamente dai dati sorgente.

### Stack/unstack

I metodi `stack` e `unstack` permettono di **riorganizzare i valori** contenuti in una serie o DataFrame **spostando i livelli degli indici**.

- `stack` sposta un livello dall'indice delle colonne a quello delle righe, per cui dati che prima erano affiancati vengono impilati (_stacked_) uno sopra l'altro.
- `unstack` al contrario sposta un livello dall'indice delle righe a quello delle colonne, rendendo affiancati dati che prima erano impilati.

In questa logica una serie è paragonabile ad un frame il cui indice delle colonne non ha livelli e c'è solamente una colonna, per cui può essere usato `unstack` per far diventare indice delle colonne un livello dell'indice delle righe.

Riprendiamo ad esempio la serie con il ricavo medio giornaliero per tipo di negozio e assortimento mostrato sopra.

In [49]:
mean_sales_by_categories = sales.groupby(["StoreType", "Assortment"])["Sales"].mean()
mean_sales_by_categories

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\1348066095.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_sales_by_categories = sales.groupby(["StoreType", "Assortment"])["Sales"].mean()


StoreType  Assortment
a          a              6417.942760
           b                      NaN
           c              7776.345052
b          a             11412.990783
           b              9717.379928
           c             17526.387097
c          a              6775.873497
           b                      NaN
           c              7170.213876
d          a              6486.686994
           b                      NaN
           c              7368.128110
Name: Sales, dtype: float64

Utilizzando `unstack`, facciamo in modo che uno dei livelli dell'indice (`StoreType` o `Assortment`) diventi l'indice delle colonne in un frame che contiene gli stessi dati della serie.

Se non è specificato nulla, il livello spostato è l'ultimo (quello più a destra), in questo caso `Assortment`.

In [50]:
mean_sales_by_categories.unstack()

Assortment,a,b,c
StoreType,,,
a,6417.942760,NaN,7776.345052
b,11412.990783,9717.379928,17526.387097
c,6775.873497,NaN,7170.213876
d,6486.686994,NaN,7368.128110


Altrimenti è possibile indicare il nome o la posizione (prima = 0) del livello da spostare). Spostando il livello `StoreType` invece di `Assortment` otteniamo in pratica la versione trasposta (righe e colonne scambiate) del frame sopra.

In [51]:
mean_sales_by_categories.unstack("StoreType")  # oppure unstack(0)

StoreType,a,b,c,d
Assortment,,,,
a,6417.942760,11412.990783,6775.873497,6486.686994
b,NaN,9717.379928,NaN,NaN
c,7776.345052,17526.387097,7170.213876,7368.128110


Se l'indice originale non possiede tutte le coppie di valori possibili di `StoreType` e `Assortment`, il frame conterrà valori mancanti. In questo caso specifico, anche se la serie originale non avesse esplicitamente valori `NaN`...

In [52]:
mean_sales_by_categories.dropna()

StoreType  Assortment
a          a              6417.942760
           c              7776.345052
b          a             11412.990783
           b              9717.379928
           c             17526.387097
c          a              6775.873497
           c              7170.213876
d          a              6486.686994
           c              7368.128110
Name: Sales, dtype: float64

...questi appariranno nel risultato di `unstack` in corrispondenza delle coppie `StoreType`/`Assortment` assenti.

In [53]:
mean_sales_by_categories.dropna().unstack()

Assortment,a,b,c
StoreType,,,
a,6417.942760,NaN,7776.345052
b,11412.990783,9717.379928,17526.387097
c,6775.873497,NaN,7170.213876
d,6486.686994,NaN,7368.128110


È anche possibile assegnare un valore alternativo a `NaN` specificandolo col parametro `fill_value`.

In [54]:
mean_sales_by_categories.dropna().unstack(fill_value=0)

Assortment,a,b,c
StoreType,,,
a,6417.942760,0.000000,7776.345052
b,11412.990783,9717.379928,17526.387097
c,6775.873497,0.000000,7170.213876
d,6486.686994,0.000000,7368.128110


Proviamo ora ad operare su un frame complesso: consideriamo la tabella con i negozi classificati come sopra, ma che include sia i totali che le medie sia dei ricavi che del numero di clienti.

In [55]:
means_by_categories = sales.groupby(["StoreType", "Assortment"])[["Sales", "Customers"]].agg(["sum", "mean"])
means_by_categories

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\1175295286.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means_by_categories = sales.groupby(["StoreType", "Assortment"])[["Sales", "Customers"]].agg(["sum", "mean"])


Sales               Customers             
                           sum          mean       sum         mean
StoreType Assortment                                               
a         a           65816003   6417.942760   7361012   717.797367
          b                  0           NaN         0          NaN
          c           46673623   7776.345052   5030022   838.057647
b         a            2476619  11412.990783    390773  1800.797235
          b            2711149   9717.379928    654229  2344.906810
          c             543318  17526.387097     94854  3059.806452
c         a           14087041   6775.873497   1652050   794.636845
          b                  0           NaN         0          NaN
          c           13745300   7170.213876   1462425   762.871674
d         a           22443937   6486.686994   1958383   566.006647
          b                  0           NaN         0          NaN
          c           43825626   7368.128110   3650140   613.675185

Con `unstack` possiamo spostare un livello dalle righe alle colonne, "allargando" il frame. Il livello spostato diventa l'ultimo livello (quello più in basso) dell'indice delle colonne.

In [56]:
means_by_categories.unstack("Assortment")

Sales                                                \
                 sum                             mean                
Assortment         a        b         c             a            b   
StoreType                                                            
a           65816003        0  46673623   6417.942760          NaN   
b            2476619  2711149    543318  11412.990783  9717.379928   
c           14087041        0  13745300   6775.873497          NaN   
d           22443937        0  43825626   6486.686994          NaN   

                         Customers                                            \
                               sum                          mean               
Assortment             c         a       b        c            a           b   
StoreType                                                                      
a            7776.345052   7361012       0  5030022   717.797367         NaN   
b           17526.387097    390773  654229    94854  1800.797235  2344.90681   
c            7170.213876   1652050       0  1462425   794.636845         NaN   
d            7368.128110   1958383       0  3650140   566.006647         NaN   

                         
                         
Assortment            c  
StoreType                
a            838.057647  
b           3059.806452  
c            762.871674  
d            613.675185

Con `stack` spostiamo invece un livello dalle colonne alle righe, "restringendo" il frame. Anche qui il livello spostato diventa l'ultimo dell'indice delle righe.

In [57]:
means_by_categories.stack(1)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\4179308832.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  means_by_categories.stack(1)


Sales     Customers
StoreType Assortment                                 
a         a          sum   6.581600e+07  7.361012e+06
                     mean  6.417943e+03  7.177974e+02
          b          sum   0.000000e+00  0.000000e+00
          c          sum   4.667362e+07  5.030022e+06
                     mean  7.776345e+03  8.380576e+02
b         a          sum   2.476619e+06  3.907730e+05
                     mean  1.141299e+04  1.800797e+03
          b          sum   2.711149e+06  6.542290e+05
                     mean  9.717380e+03  2.344907e+03
          c          sum   5.433180e+05  9.485400e+04
                     mean  1.752639e+04  3.059806e+03
c         a          sum   1.408704e+07  1.652050e+06
                     mean  6.775873e+03  7.946368e+02
          b          sum   0.000000e+00  0.000000e+00
          c          sum   1.374530e+07  1.462425e+06
                     mean  7.170214e+03  7.628717e+02
d         a          sum   2.244394e+07  1.958383e+06
                     mean  6.486687e+03  5.660066e+02
          b          sum   0.000000e+00  0.000000e+00
          c          sum   4.382563e+07  3.650140e+06
                     mean  7.368128e+03  6.136752e+02

Se si vogliono i livelli degli indici ordinati diversamente, è possibile usare il metodo `reorder_levels` passando la lista ordinata delle posizioni originali degli indici. Dal risultato sopra spostiamo ad esempio l'ultimo livello dell'indice delle righe (`sum`/`mean`) a sinistra.

In [58]:
means_by_categories.stack(1).reorder_levels([2, 0, 1], axis=0)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\3022490868.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  means_by_categories.stack(1).reorder_levels([2, 0, 1], axis=0)


Sales     Customers
     StoreType Assortment                            
sum  a         a           6.581600e+07  7.361012e+06
mean a         a           6.417943e+03  7.177974e+02
sum  a         b           0.000000e+00  0.000000e+00
               c           4.667362e+07  5.030022e+06
mean a         c           7.776345e+03  8.380576e+02
sum  b         a           2.476619e+06  3.907730e+05
mean b         a           1.141299e+04  1.800797e+03
sum  b         b           2.711149e+06  6.542290e+05
mean b         b           9.717380e+03  2.344907e+03
sum  b         c           5.433180e+05  9.485400e+04
mean b         c           1.752639e+04  3.059806e+03
sum  c         a           1.408704e+07  1.652050e+06
mean c         a           6.775873e+03  7.946368e+02
sum  c         b           0.000000e+00  0.000000e+00
               c           1.374530e+07  1.462425e+06
mean c         c           7.170214e+03  7.628717e+02
sum  d         a           2.244394e+07  1.958383e+06
mean d         a           6.486687e+03  5.660066e+02
sum  d         b           0.000000e+00  0.000000e+00
               c           4.382563e+07  3.650140e+06
mean d         c           7.368128e+03  6.136752e+02

Possiamo da qui usare `sort_index` per avere prima tutte le medie e poi tutte le somme.

In [59]:
means_by_categories.stack(1).reorder_levels([2, 0, 1], axis=0).sort_index()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\616193287.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  means_by_categories.stack(1).reorder_levels([2, 0, 1], axis=0).sort_index()


Sales     Customers
     StoreType Assortment                            
mean a         a           6.417943e+03  7.177974e+02
               c           7.776345e+03  8.380576e+02
     b         a           1.141299e+04  1.800797e+03
               b           9.717380e+03  2.344907e+03
               c           1.752639e+04  3.059806e+03
     c         a           6.775873e+03  7.946368e+02
               c           7.170214e+03  7.628717e+02
     d         a           6.486687e+03  5.660066e+02
               c           7.368128e+03  6.136752e+02
sum  a         a           6.581600e+07  7.361012e+06
               b           0.000000e+00  0.000000e+00
               c           4.667362e+07  5.030022e+06
     b         a           2.476619e+06  3.907730e+05
               b           2.711149e+06  6.542290e+05
               c           5.433180e+05  9.485400e+04
     c         a           1.408704e+07  1.652050e+06
               b           0.000000e+00  0.000000e+00
               c           1.374530e+07  1.462425e+06
     d         a           2.244394e+07  1.958383e+06
               b           0.000000e+00  0.000000e+00
               c           4.382563e+07  3.650140e+06

### Creazione tabelle pivot

Abbiamo visto sopra come applicando opportunamente sequenze di operazioni che partono da `groupby` e finiscono con `stack` e `unstack` possiamo costruire tabelle pivot personalizzando cosa è rappresentato in ciascuna riga e ciascuna colonna.

Possiamo però ottenere un risultato simile in modo più diretto usando il metodo `pivot_table`, che permette di creare rapidamente una tabella pivot da un frame specificando:

- le colonne del frame sorgente da usare come valori (`values`, default: tutte le colonne numeriche);
- le colonne del frame da usare come livelli di righe (`index`) e di colonne (`columns`) della tabella pivot;
- una o più funzioni con cui aggregare i dati (`aggfunc`, default: "mean", media), sono valide le stesse utilizzabili in `agg`.

Ad esempio un modo alternativo per ottenere un frame simile a `means_by_categories` (solo con gli indici delle colonne scambiati) possiamo scrivere:

In [60]:
sales.pivot_table(
    values=["Sales", "Customers"],      # ricavi e numero clienti
    index=["StoreType", "Assortment"],  # per tipologia di negozio
    columns=[],
    aggfunc=["sum", "mean"],            # totali e medi giornalieri
)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2369606546.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sales.pivot_table(
C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2369606546.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sales.pivot_table(


sum                   mean              
                     Customers     Sales    Customers         Sales
StoreType Assortment                                               
a         a            7361012  65816003   717.797367   6417.942760
          b                  0         0          NaN           NaN
          c            5030022  46673623   838.057647   7776.345052
b         a             390773   2476619  1800.797235  11412.990783
          b             654229   2711149  2344.906810   9717.379928
          c              94854    543318  3059.806452  17526.387097
c         a            1652050  14087041   794.636845   6775.873497
          b                  0         0          NaN           NaN
          c            1462425  13745300   762.871674   7170.213876
d         a            1958383  22443937   566.006647   6486.686994
          b                  0         0          NaN           NaN
          c            3650140  43825626   613.675185   7368.128110

Come altro esempio, estraiamo una tabella dove ogni riga è un giorno della settimana, ogni colonna uno `StoreType` e i valori sono i ricavi medi giornalieri nei rispettivi giorni e tipologie.

In [61]:
sales.pivot_table(
    values=["Sales"],       # ricavi giornalieri
    index=["DayOfWeek"],    # per giorno della settimana
    columns=["StoreType"],  # per tipologia di negozio
    aggfunc=["mean"],       # medi
)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\1313940940.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sales.pivot_table(


mean                                        
                 Sales                                        
StoreType            a             b            c            d
DayOfWeek                                                     
Fri        7101.518161  11331.023529  7057.268919  7332.487931
Mon        8117.860891  11258.102941  8061.086149  8189.407328
Sat        5009.462724   8267.279412  6029.925676  6197.798132
Sun        3969.000000  12913.132353          NaN  4833.666667
Thu        7036.472842  10813.000000  6877.655405  6965.083333
Tue        7095.372345  10542.985294  6850.005068  6750.624282
Wed        7097.016650  10895.317647  6923.533784  6844.785057

Aggiungendo l'opzione `margins=True` a `pivot_table` possiamo inoltre aggiungere una riga e una colonna di aggregazioni parziali sui gruppi delle singole righe e colonne. In pratica in questo caso possiamo aggiungere una riga con le medie complessive per tipo di negozio e una colonna con le medie complessive per giorno della settimana. Con `margins_name` indichiamo l'etichetta applicata alle due (default: `All`).

In [62]:
sales.pivot_table(
    values=["Sales"],       # ricavi giornalieri
    index=["DayOfWeek"],    # per giorno della settimana
    columns=["StoreType"],  # per tipologia di negozio
    aggfunc=["mean"],       # medi
    margins=True,
    margins_name="Tutti",
)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2310522015.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sales.pivot_table(


mean                                                     
                 Sales                                                     
StoreType            a             b            c            d        Tutti
DayOfWeek                                                                  
Fri        7101.518161  11331.023529  7057.268919  7332.487931  7232.429213
Mon        8117.860891  11258.102941  8061.086149  8189.407328  8180.631709
Sat        5009.462724   8267.279412  6029.925676  6197.798132  5566.347631
Sun        3969.000000  12913.132353          NaN  4833.666667  8764.170543
Thu        7036.472842  10813.000000  6877.655405  6965.083333  7050.713259
Tue        7095.372345  10542.985294  6850.005068  6750.624282  7007.631709
Wed        7097.016650  10895.317647  6923.533784  6844.785057  7053.122126
Tutti      6919.457834  10874.925996  6965.050300  7043.958652  7033.344905

Il valore in basso a destra è in pratica la media globale di `Sales` su tutta la tabella.

In [63]:
sales["Sales"].mean()

7033.344905260368

### Esercizio 4: Pivoting

- **(4a)** Usando `groupby` e `stack`/`unstack`, indicare in una tabella con una riga per giorno della settimana e due colonne che indicano se era applicata o meno una promozione (`Promo`) il numero di clienti giornaliero medio in ciascun caso. _(simile a esercizio 3a ma cambia la forma dei risultati)_
- **(4b)** Ottenere la stessa tabella dell'esercizio 4a usando `pivot_table`
- **(4c)** Usando `stack`/`unstack` ottenere dal frame `sales` una grande tabella con una riga per ogni data e una colonna per ogni negozio che indichi il ricavo giornaliero per ogni data e negozio
- **(4d)** Partendo dal risultato dell'esercizio precedente, contare quante volte ciascun negozio è stato quello con più ricavi in una certa data _(stesso risultato dell'esercizio 3d)_

In [68]:
# 4a
sales.groupby(["DayOfWeek", "Promo"])["Customers"].mean().unstack()

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\2450242060.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales.groupby(["DayOfWeek", "Promo"])["Customers"].mean().unstack()


Promo,False,True
DayOfWeek,,
Fri,697.350696,801.712189
Mon,749.848226,887.443845
Sat,606.321356,NaN
Sun,1458.480620,NaN
Thu,693.390211,776.031746
Tue,665.662775,803.277628
Wed,665.228559,779.160431


In [65]:
# 4b
sales.pivot_table(
    values=["Customers"],
    index=["DayOfWeek"],
    columns=["Promo"],
    aggfunc=["mean"],
)

C:\Users\marti\AppData\Local\Temp\ipykernel_12332\745798396.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sales.pivot_table(


mean            
             Customers            
Promo            False       True 
DayOfWeek                         
Fri         697.350696  801.712189
Mon         749.848226  887.443845
Sat         606.321356         NaN
Sun        1458.480620         NaN
Thu         693.390211  776.031746
Tue         665.662775  803.277628
Wed         665.228559  779.160431

In [66]:
# 4c
sales["Sales"].unstack()

Store,1,2,3,4,5,6,7,8,9,10,...,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-01,5223.0,6431.0,8311.0,11216.0,6253.0,5804.0,11024.0,8523.0,8490.0,7896.0,...,5411.0,5738.0,7086.0,4294.0,6604.0,3701.0,10620.0,8222.0,27071.0,7701.0
2015-07-02,5558.0,6189.0,7450.0,10323.0,6035.0,5335.0,10996.0,8658.0,8395.0,6317.0,...,5256.0,5652.0,7365.0,4313.0,5829.0,3945.0,7447.0,7932.0,24898.0,6858.0
2015-07-03,4665.0,4838.0,8149.0,11543.0,4475.0,4594.0,11976.0,6535.0,8363.0,5915.0,...,5711.0,5284.0,6351.0,5181.0,4953.0,4596.0,9285.0,6942.0,23750.0,7412.0
2015-07-04,4797.0,4475.0,4482.0,9358.0,2240.0,2979.0,6559.0,3418.0,6223.0,5120.0,...,3035.0,4505.0,2942.0,3537.0,2895.0,2594.0,5837.0,5340.0,18529.0,6543.0
2015-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-06,4359.0,4132.0,6736.0,9967.0,4328.0,4508.0,10790.0,6740.0,7758.0,6143.0,...,4935.0,4157.0,5981.0,3789.0,4033.0,4256.0,7841.0,6631.0,23004.0,6501.0
2015-07-07,3650.0,4313.0,6180.0,8127.0,4036.0,4021.0,9207.0,5365.0,6940.0,5476.0,...,4620.0,3877.0,5171.0,3648.0,3892.0,3209.0,5825.0,5836.0,18466.0,5138.0
2015-07-08,3797.0,5676.0,6347.0,8620.0,4990.0,4072.0,9204.0,5322.0,5621.0,4865.0,...,4517.0,4147.0,5382.0,3886.0,3510.0,3530.0,7515.0,6361.0,20806.0,5900.0
2015-07-09,3897.0,4082.0,6613.0,9093.0,4601.0,4095.0,9608.0,5602.0,6285.0,4871.0,...,4550.0,4163.0,6025.0,3903.0,3876.0,3828.0,7026.0,6226.0,20355.0,5686.0


In [67]:
# 4d
sales["Sales"].unstack().idxmax(axis=1).value_counts()

1114    18
842      6
262      4
876      2
817      1
Name: count, dtype: int64